# Hydroelectric Power Generation

In [124]:
%reset
import numpy as np
import cvxpy as cvx

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Part 1

In [125]:
K = 3
L = 2
J = 3
w_l = np.array([20, 15])
lam = np.array([10, 30, 8])
rho = np.array([30./47, 43./58, 50./72])
V_0 = np.array([120, 130])
V_min = np.zeros((L, K))
V_max = np.array([[95000, 95000, 95000], [11000, 11000, 11000]])
A = np.array([[15, 15, 15], [12, 12, 12]])
T_max = np.array([[55, 55, 55], [65, 65, 65], [80, 80, 80]])
T_min = np.zeros((J, K))

In [126]:
def solve(K, L, J, w_l, lam, rho, V_0, V_min, V_max, A, T_min, T_max):
    """Solve the optimization problem described in part 1 using a solver of your choice.
    
    Args:
        K (int): number of timestesp
        L (int): number of reservoirs
        J (int): number of turbines
        etc.
    
    Returns:
        the optimal water volumes, T_opt
        the optimal objective value, obj_val
    """
    rho_tiled = np.reshape(np.repeat(rho, K), (J, K))
    lam_tiled = np.tile(lam, (J, 1))
    rho_lam = np.multiply(rho_tiled, lam_tiled)
    V = cvx.Variable((L,K))
    T = cvx.Variable((J,K))
    obj = cvx.Minimize(V[:,0] * w_l - V[:,K-1] * w_l - cvx.sum(cvx.multiply(rho_lam, T)))
    constraints = [T >= np.zeros((J,K)), T <= T_max, V >= V_min, V <= V_max,
                   V[0, 0] == V_0[0] + A[0, 0] - (T[0, 0] + T[1, 0]),
                   V[1, 0] == V_0[1] + A[1, 0] + (T[0, 0] + T[1, 0]) - T[2, 0],
                   V[0, 1] == V[0, 0] + A[0, 1] - (T[0, 1] + T[1, 1]),
                   V[1, 1] == V[1, 0] + A[1, 1] + (T[0, 1] + T[1, 1]) - T[2, 1],
                   V[0, 2] == V[0, 1] + A[0, 2] - (T[0, 2] + T[1, 2]),
                   V[1, 2] == V[1, 1] + A[1, 2] + (T[0, 2] + T[1, 2]) - T[2, 2]
                  ]
    prob = cvx.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value.
    obj_val = prob.value
    T_opt = np.array(T.value)
    return T_opt, obj_val

In [127]:
# Solve the optimization problem
T_opt, obj_val = solve(K, L, J, w_l, lam, rho, V_0, V_min, V_max, A, T_min, T_max)

# Round values for readability (optional)
obj_val = np.around(obj_val, decimals=7)
T_opt = np.around(T_opt, decimals=0)

# Print results
print("Optimal value:\n{}\n".format(obj_val))
print("Optimal water volumes, T_opt:\n{}".format(T_opt))

Optimal value:
-4117.4826773

Optimal water volumes, T_opt:
[[ 0. 55.  0.]
 [30. 65. 15.]
 [80. 80. -0.]]


## Part 2 (Don't forget to re-initialize variables!)

In [177]:
K = 3
L = 2
J = 3
w_l = np.array([20, 15])
lam = np.array([12, 30, 4])
rho = np.array([30./47, 43./58, 50./72])
V_0 = np.array([120, 130])
V_min = np.zeros((L, K))
V_max = np.array([[95000, 95000, 95000], [11000, 11000, 11000]])
A = np.array([[15, 15, 15], [12, 12, 12]])
T_min = np.zeros((J, K))
T_max = np.array([[55, 55, 55], [65, 65, 65], [80, 80, 80]])

In [263]:
def solve_regularized(K, L, J, T_hat, w_l, lam, rho, V_0, V_min, V_max, A, T_min, T_max, gamma):
    """Solve the optimization problem described in part 2 using a solver of your choice.
    
    Args:
        T_hat (numpy.ndarray): optimal solution T_opt from part 1
        gamma (float): regularization multiplier
        (remaining variables have same description as in part 1)
    
    Returns:
        the optimal water volumes, T_opt_reg
        the optimal objective value, obj_val_reg
    """
    rho_tiled = np.reshape(np.repeat(rho, K), (J, K))
    lam_tiled = np.tile(lam, (J, 1))
    rho_lam = np.multiply(rho_tiled, lam_tiled)
    V = cvx.Variable((L,K))
    T = cvx.Variable((J,K))
    obj = cvx.Minimize(V[:,0] * w_l - V[:,K-1] * w_l - cvx.sum(cvx.multiply(rho_lam, T)) +
                       gamma * cvx.sum(
                           [cvx.max(cvx.abs(T[j] - T_hat[j])) for j in range(J)]
                       ))
    constraints = [T >= np.zeros((J,K)), T <= T_max, V >= V_min, V <= V_max,
                   V[0, 0] == V_0[0] + A[0, 0] - (T[0, 0] + T[1, 0]),
                   V[1, 0] == V_0[1] + A[1, 0] + (T[0, 0] + T[1, 0]) - T[2, 0],
                   V[0, 1] == V[0, 0] + A[0, 1] - (T[0, 1] + T[1, 1]),
                   V[1, 1] == V[1, 0] + A[1, 1] + (T[0, 1] + T[1, 1]) - T[2, 1],
                   V[0, 2] == V[0, 1] + A[0, 2] - (T[0, 2] + T[1, 2]),
                   V[1, 2] == V[1, 1] + A[1, 2] + (T[0, 2] + T[1, 2]) - T[2, 2]
                  ]
    prob = cvx.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value.
    obj_val_reg = prob.value
    T_opt_reg = np.array(T.value)
    return T_opt_reg, obj_val_reg

In [265]:
# Solve the optimization problem
T_opt_reg, obj_val_reg = solve_regularized(K, L, J, T_opt, w_l, lam, rho, V_0, V_min, V_max, A, T_min, T_max, gamma=np.power(10, 0, dtype=np.float32))

# Round values for readability (optional)
obj_val_reg = np.around(obj_val_reg, decimals=7)
T_opt_reg = np.around(T_opt_reg, decimals=0)

# Print results
print("Optimal value:\n{}\n".format(obj_val_reg))
print("Optimal water volumes, T_opt_reg:\n{}".format(T_opt_reg))

Optimal value:
-4244.1110293

Optimal water volumes, T_opt_reg:
[[ 0. 55.  0.]
 [30. 65.  0.]
 [80. 80. -0.]]
